In [35]:
# Imports
from collections import Counter
import os
import torch
import torch.nn as nn
from transformers import AutoTokenizer
import uuid
import time
import numpy as np
import matplotlib as plt
import json
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
from tqdm import tqdm

In [36]:
# get keys
OPENA_AI_API_KEY = os.getenv('OPENA_AI_API_KEY')
OPENA_AI_ORG_ID = os.getenv('OPENA_AI_ORG_ID')

GPT_MODEL = "gpt-4"

def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))





In [37]:
explanation = '''You are responsible for managing a tldraw canvas shape editor with abilities to manipulate shapes and the editor's state. You can create, modify, delete shapes, and control the internal state..'''

editor_explanaton = '''Manage the internal state, make document changes, or respond to changes.'''

editor_functions = [
    'create_shape',
    'update_shape',
]


editor_functions_docs = [
    'Create a shape on the canvas.',
    'Update a shape on the canvas.',
]

shapes_explanation = '''Visual elements on the canvas.'''
shapes = [
    'draw',
    'text',
    'arrow',
    'geo',
]
shape_docs = [
    'Create shapes with custom paths.',
    'Add text.',
    'Draw an arrow between points',
    'Create geometric shapes',
]


editor_functions_args = [
    {
    "type": "object",
    "properties": {
      "x": {
        "type": "number",
        "description": "x-coordinate of the shape"
      },
      "y": {
        "type": "number",
        "description": "y-coordinate of the shape"
      },
      "rotation": {
        "type": "number",
        "description": "rotation of the shape"
      },
      "id": {
        "type": "string",
        "description": "Unique identifier for the shape"
      },
      "type": {
        "type": "string",
        "enum": shapes,
        "description": "Type of the shape, e.g. geo"
      },
      "props": {
        "type": "object",
        "properties": {
          "w": {
            "type": "number",
            "description": "Width of the shape"
          },
          "h": {
            "type": "number",
            "description": "Height of the shape"
          },
          "geo": {
            "type": "string",
            "enum": ["rectangle", "circle"],
            "description": "Geometric shape, e.g. rectangle"
          },
          "text": {
            "type": "string",
            "description": "Text to display on the shape"
          }
        },
      }
    },
} for i in range(len(editor_functions))
]





functions = [
    {
        "name": editor_functions[i],
        "description": editor_functions_docs[i],
        "parameters": editor_functions_args[i],
    } for i in range(len(editor_functions))
]


shapes_list = [
    {
        "name": shapes[i],
        "description": shape_docs[i],
    } for i in range(len(shapes))
]
functions_str = '\n    '.join([str(function) for function in functions])
shapes_str = '\n    '.join([str(shape) for shape in shapes_list])


system_prompt = f'''{explanation}
- Editor: {editor_explanaton}
- Shapes: {shapes_explanation}
  - Shapes:
    {shapes_str}
Every prompt will include the current state and user's request. Response with the sequence of function calls to perform the user's request.
'''

print(system_prompt)

print('---------')

print(functions)

You are responsible for managing a tldraw canvas shape editor with abilities to manipulate shapes and the editor's state. You can create, modify, delete shapes, and control the internal state..
- Editor: Manage the internal state, make document changes, or respond to changes.
- Shapes: Visual elements on the canvas.
  - Shapes:
    {'name': 'draw', 'description': 'Create shapes with custom paths.'}
    {'name': 'text', 'description': 'Add text.'}
    {'name': 'arrow', 'description': 'Draw an arrow between points'}
    {'name': 'geo', 'description': 'Create geometric shapes'}
Every prompt will include the current state and user's request. Response with the sequence of function calls to perform the user's request.

---------
[{'name': 'create_shape', 'description': 'Create a shape on the canvas.', 'parameters': {'type': 'object', 'properties': {'x': {'type': 'number', 'description': 'x-coordinate of the shape'}, 'y': {'type': 'number', 'description': 'y-coordinate of the shape'}, 'rotati

In [38]:
current_state = {
    "screen_bounds": {
        "x": 0,
        "y": 0,
        "w": 1512,
        "h": 610,
    },
    "shapes": [
    {
        "x": 391.20834875401147,
        "y": 237.18911086754463,
        "rotation": 5.759586531581287,
        "id": "shape:shape-0",
        "type": "geo",
        "props": {
            "w": 130,
            "h": 70,
            "geo": "rectangle",
            "text": "display-0",
        },
    },
    {
        "x": 262.8108891324554,
        "y": 161.20834875401147,
        "id": "shape:shape-1",
        "type": "geo",
        "props": {
            "w": 130,
            "h": 70,
            "geo": "rectangle",
            "color": "blue",
            "text": "display-1",
        },
    },
    {
        "x": 630,
        "y": 170,
        "rotation": 3.141592653589793,
        "id": "shape:shape-2",
        "type": "geo",
        "props": {
            "w": 130,
            "h": 70,
            "geo": "rectangle",
            "text": "display-2",
        },
    }
],
"selected_shape_ids": [],
}

def get_current_state(current_state):
    return f'''Current state: {current_state}'''

print(get_current_state(current_state))

Current state: {'screen_bounds': {'x': 0, 'y': 0, 'w': 1512, 'h': 610}, 'shapes': [{'x': 391.20834875401147, 'y': 237.18911086754463, 'rotation': 5.759586531581287, 'id': 'shape:shape-0', 'type': 'geo', 'props': {'w': 130, 'h': 70, 'geo': 'rectangle', 'text': 'display-0'}}, {'x': 262.8108891324554, 'y': 161.20834875401147, 'id': 'shape:shape-1', 'type': 'geo', 'props': {'w': 130, 'h': 70, 'geo': 'rectangle', 'color': 'blue', 'text': 'display-1'}}, {'x': 630, 'y': 170, 'rotation': 3.141592653589793, 'id': 'shape:shape-2', 'type': 'geo', 'props': {'w': 130, 'h': 70, 'geo': 'rectangle', 'text': 'display-2'}}], 'selected_shape_ids': []}


In [39]:
user_prompt = '''Align all of the shapes horizontally in a row, with all text facing upright.'''

In [40]:
messages = []
messages.append({"role": "system", "content": system_prompt})
messages.append({"role": "system", "content": get_current_state(current_state)})
messages.append({"role": "user", "content": user_prompt})


In [41]:
for i in functions:
    print(i)


{'name': 'create_shape', 'description': 'Create a shape on the canvas.', 'parameters': {'type': 'object', 'properties': {'x': {'type': 'number', 'description': 'x-coordinate of the shape'}, 'y': {'type': 'number', 'description': 'y-coordinate of the shape'}, 'rotation': {'type': 'number', 'description': 'rotation of the shape'}, 'id': {'type': 'string', 'description': 'Unique identifier for the shape'}, 'type': {'type': 'string', 'enum': ['draw', 'text', 'arrow', 'geo'], 'description': 'Type of the shape, e.g. geo'}, 'props': {'type': 'object', 'properties': {'w': {'type': 'number', 'description': 'Width of the shape'}, 'h': {'type': 'number', 'description': 'Height of the shape'}, 'geo': {'type': 'string', 'enum': ['rectangle', 'circle'], 'description': 'Geometric shape, e.g. rectangle'}, 'text': {'type': 'string', 'description': 'Text to display on the shape'}}}}}}
{'name': 'update_shape', 'description': 'Update a shape on the canvas.', 'parameters': {'type': 'object', 'properties': 

In [42]:
response = openai.ChatCompletion.create(
  model=GPT_MODEL,
  messages=messages,
  temperature=0.2,
  functions=functions,
 )

In [43]:
print(response)

{
  "id": "chatcmpl-7nfd9JiwW5yQC8MFSmq1bVygF6Nd0",
  "object": "chat.completion",
  "created": 1692072455,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "[\n  {\"function\": \"functions.update_shape\", \"args\": {\"id\": \"shape:shape-0\", \"x\": 200, \"y\": 300, \"rotation\": 0}},\n  {\"function\": \"functions.update_shape\", \"args\": {\"id\": \"shape:shape-1\", \"x\": 400, \"y\": 300, \"rotation\": 0}},\n  {\"function\": \"functions.update_shape\", \"args\": {\"id\": \"shape:shape-2\", \"x\": 600, \"y\": 300, \"rotation\": 0}}\n]"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 710,
    "completion_tokens": 123,
    "total_tokens": 833
  }
}


In [44]:
assistant_message = response["choices"][0]["message"]
print(assistant_message)

messages.append(assistant_message)

{
  "role": "assistant",
  "content": "[\n  {\"function\": \"functions.update_shape\", \"args\": {\"id\": \"shape:shape-0\", \"x\": 200, \"y\": 300, \"rotation\": 0}},\n  {\"function\": \"functions.update_shape\", \"args\": {\"id\": \"shape:shape-1\", \"x\": 400, \"y\": 300, \"rotation\": 0}},\n  {\"function\": \"functions.update_shape\", \"args\": {\"id\": \"shape:shape-2\", \"x\": 600, \"y\": 300, \"rotation\": 0}}\n]"
}


In [45]:
pretty_print_conversation(messages)


system: You are responsible for managing a tldraw canvas shape editor with abilities to manipulate shapes and the editor's state. You can create, modify, delete shapes, and control the internal state..
- Editor: Manage the internal state, make document changes, or respond to changes.
- Shapes: Visual elements on the canvas.
  - Shapes:
    {'name': 'draw', 'description': 'Create shapes with custom paths.'}
    {'name': 'text', 'description': 'Add text.'}
    {'name': 'arrow', 'description': 'Draw an arrow between points'}
    {'name': 'geo', 'description': 'Create geometric shapes'}
Every prompt will include the current state and user's request. Response with the sequence of function calls to perform the user's request.


system: Current state: {'screen_bounds': {'x': 0, 'y': 0, 'w': 1512, 'h': 610}, 'shapes': [{'x': 391.20834875401147, 'y': 237.18911086754463, 'rotation': 5.759586531581287, 'id': 'shape:shape-0', 'type': 'geo', 'props': {'w': 130, 'h': 70, 'geo': 'rectangle', 'text': 

In [46]:
pretty_print_conversation(messages)


system: You are responsible for managing a tldraw canvas shape editor with abilities to manipulate shapes and the editor's state. You can create, modify, delete shapes, and control the internal state..
- Editor: Manage the internal state, make document changes, or respond to changes.
- Shapes: Visual elements on the canvas.
  - Shapes:
    {'name': 'draw', 'description': 'Create shapes with custom paths.'}
    {'name': 'text', 'description': 'Add text.'}
    {'name': 'arrow', 'description': 'Draw an arrow between points'}
    {'name': 'geo', 'description': 'Create geometric shapes'}
Every prompt will include the current state and user's request. Response with the sequence of function calls to perform the user's request.


system: Current state: {'screen_bounds': {'x': 0, 'y': 0, 'w': 1512, 'h': 610}, 'shapes': [{'x': 391.20834875401147, 'y': 237.18911086754463, 'rotation': 5.759586531581287, 'id': 'shape:shape-0', 'type': 'geo', 'props': {'w': 130, 'h': 70, 'geo': 'rectangle', 'text': 